In [45]:
!pip install xgboost

  Obtaining dependency information for xgboost from https://files.pythonhosted.org/packages/24/ec/ad387100fa3cc2b9b81af0829b5ecfe75ec5bb19dd7c19d4fea06fb81802/xgboost-2.0.3-py3-none-win_amd64.whl.metadata
   ---------------------------------------- 0.0/99.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/99.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/99.8 MB 435.7 kB/s eta 0:03:49
   ---------------------------------------- 0.1/99.8 MB 930.9 kB/s eta 0:01:48
   ---------------------------------------- 0.3/99.8 MB 1.7 MB/s eta 0:00:59
   ---------------------------------------- 0.6/99.8 MB 2.8 MB/s eta 0:00:36
   ---------------------------------------- 0.8/99.8 MB 3.4 MB/s eta 0:00:29
    --------------------------------------- 1.3/99.8 MB 4.7 MB/s eta 0:00:21
    --------------------------------------- 1.7/99.8 MB 4.9 MB/s eta 0:00:20
    --------------------------------------- 2.2/99.8 MB 5.5 MB/s eta 0:00:18
   - ------------------------------

In [47]:
!pip install scikit-learn

In [127]:
!pip install lightgbm

  Obtaining dependency information for lightgbm from https://files.pythonhosted.org/packages/e1/4c/4685ccfae9806f561de716e32549190c1f533dde5bcadaf83bdf23972cf0/lightgbm-4.3.0-py3-none-win_amd64.whl.metadata
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.3 MB 653.6 kB/s eta 0:00:02
   --- ------------------------------------ 0.1/1.3 MB 939.4 kB/s eta 0:00:02
   ----- ---------------------------------- 0.2/1.3 MB 1.2 MB/s eta 0:00:01
   ---------- ----------------------------- 0.3/1.3 MB 1.6 MB/s eta 0:00:01
   --------------- ------------------------ 0.5/1.3 MB 2.0 MB/s eta 0:00:01
   ------------------------ --------------- 0.8/1.3 MB 2.8 MB/s eta 0:00:01
   --------------------------------- ------ 1.1/1.3 MB 3.2 MB/s eta 0:00:01
   ---------------------------------------- 1.3/1.3 MB 3.5 MB/s eta 0:00:00


In [23]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
###############################################
import warnings
warnings.filterwarnings("ignore")

In [24]:
pd.set_option('display.max_columns', None)

In [33]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")
df_train.head()

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,0,Male,24.443011,1.699998,81.669950,yes,yes,2.000000,2.983297,Sometimes,no,2.763573,no,0.000000,0.976473,Sometimes,Public_Transportation,Overweight_Level_II
1,1,Female,18.000000,1.560000,57.000000,yes,yes,2.000000,3.000000,Frequently,no,2.000000,no,1.000000,1.000000,no,Automobile,Normal_Weight
2,2,Female,18.000000,1.711460,50.165754,yes,yes,1.880534,1.411685,Sometimes,no,1.910378,no,0.866045,1.673584,no,Public_Transportation,Insufficient_Weight
3,3,Female,20.952737,1.710730,131.274851,yes,yes,3.000000,3.000000,Sometimes,no,1.674061,no,1.467863,0.780199,Sometimes,Public_Transportation,Obesity_Type_III
4,4,Male,31.641081,1.914186,93.798055,yes,yes,2.679664,1.971472,Sometimes,no,1.979848,no,1.967973,0.931721,Sometimes,Public_Transportation,Overweight_Level_II


In [34]:
from sklearn import preprocessing 
from sklearn.model_selection import train_test_split
def data_transformation(df):
    df['family_history_with_overweight'] = df['family_history_with_overweight'].map({'yes': 1, 'no': 0})
    df['FAVC'] = df['FAVC'].map({'yes': 1, 'no': 0})
    df['SMOKE'] = df['SMOKE'].map({'yes': 1, 'no': 0})
    df['SCC'] = df['SCC'].map({'yes': 1, 'no': 0})
    df['Gender'] = df['Gender'].map({'Male': 1, 'Female': 0})
    label_encoder = preprocessing.LabelEncoder() 
    df['CAEC']= label_encoder.fit_transform(df['CAEC']) 
    df['CALC'] = label_encoder.fit_transform(df['CALC'])
    one_hot_encoder = preprocessing.OneHotEncoder()
    means_of_trns_encoded = pd.DataFrame(
        one_hot_encoder.fit_transform(df[['MTRANS']]).toarray(),
        columns=one_hot_encoder.get_feature_names_out(['MTRANS'])) 
    transformed_df = df.join(means_of_trns_encoded) 
    transformed_df = transformed_df.drop(["MTRANS"], axis=1)
    return transformed_df

In [35]:
df = data_transformation(df_train)
df_test_up = data_transformation(df_test)

In [36]:
df.head()

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,NObeyesdad,MTRANS_Automobile,MTRANS_Bike,MTRANS_Motorbike,MTRANS_Public_Transportation,MTRANS_Walking
0,0,1,24.443011,1.699998,81.669950,1,1,2.000000,2.983297,2,0,2.763573,0,0.000000,0.976473,1,Overweight_Level_II,0.0,0.0,0.0,1.0,0.0
1,1,0,18.000000,1.560000,57.000000,1,1,2.000000,3.000000,1,0,2.000000,0,1.000000,1.000000,2,Normal_Weight,1.0,0.0,0.0,0.0,0.0
2,2,0,18.000000,1.711460,50.165754,1,1,1.880534,1.411685,2,0,1.910378,0,0.866045,1.673584,2,Insufficient_Weight,0.0,0.0,0.0,1.0,0.0
3,3,0,20.952737,1.710730,131.274851,1,1,3.000000,3.000000,2,0,1.674061,0,1.467863,0.780199,1,Obesity_Type_III,0.0,0.0,0.0,1.0,0.0
4,4,1,31.641081,1.914186,93.798055,1,1,2.679664,1.971472,2,0,1.979848,0,1.967973,0.931721,1,Overweight_Level_II,0.0,0.0,0.0,1.0,0.0


In [37]:
def create_new_features(df):
    df['BMI'] = df['Weight'] / (df['Height']) ** 2
    df['Hydration_Ratio'] = (df['CH2O'] / df['Weight'])*100
    df['Technology_Usage_Ratio'] = (df['TUE'] / df['Age'])*100
    df['Alcohol_Consumption_Ratio'] = (df['CALC'] / df['Age'])*100
    df["Healthy_Diet_Score"] = df["FCVC"] * df["NCP"]
    df["BMIoncp"]=df["BMI"]/df["NCP"]
    df["BMIP_faf"] = (df["BMI"]/25)*df["FAF"]

In [38]:
create_new_features(df)
create_new_features(df_test_up)

In [39]:
df_test_up.head()

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS_Automobile,MTRANS_Bike,MTRANS_Motorbike,MTRANS_Public_Transportation,MTRANS_Walking,BMI,Hydration_Ratio,Technology_Usage_Ratio,Alcohol_Consumption_Ratio,Healthy_Diet_Score,BMIoncp,BMIP_faf
0,20758,1,26.899886,1.848294,120.644178,1,1,2.938616,3.000000,2,0,2.825629,0,0.855400,0.000000,2,0.0,0.0,0.0,1.0,0.0,35.315411,2.342118,0.000000,7.434976,8.815848,11.771804,1.208352
1,20759,0,21.000000,1.600000,66.000000,1,1,2.000000,1.000000,2,0,3.000000,0,1.000000,0.000000,2,0.0,0.0,0.0,1.0,0.0,25.781250,4.545455,0.000000,9.523810,2.000000,25.781250,1.031250
2,20760,0,26.000000,1.643355,111.600553,1,1,3.000000,3.000000,2,0,2.621877,0,0.000000,0.250502,2,0.0,0.0,0.0,1.0,0.0,41.324115,2.349341,0.963469,7.692308,9.000000,13.774705,0.000000
3,20761,1,20.979254,1.553127,103.669116,1,1,2.000000,2.977909,2,0,2.786417,0,0.094851,0.000000,2,0.0,0.0,0.0,1.0,0.0,42.976937,2.687799,0.000000,9.533227,5.955818,14.431917,0.163056
4,20762,0,26.000000,1.627396,104.835346,1,1,3.000000,3.000000,2,0,2.653531,0,0.000000,0.741069,2,0.0,0.0,0.0,1.0,0.0,39.584143,2.531142,2.850265,7.692308,9.000000,13.194714,0.000000


In [40]:
df["NObeyesdad"].unique()

array(['Overweight_Level_II', 'Normal_Weight', 'Insufficient_Weight',
       'Obesity_Type_III', 'Obesity_Type_II', 'Overweight_Level_I',
       'Obesity_Type_I'], dtype=object)

In [41]:
df['NObeyesdad'] = df['NObeyesdad'].map({'Insufficient_Weight': 0, 'Normal_Weight': 1, 'Overweight_Level_I':2, 'Overweight_Level_II':3, 'Obesity_Type_I':4 , 'Obesity_Type_II':5, 'Obesity_Type_III':6})

In [42]:
X = df.drop(["NObeyesdad"], axis=1)
y = df["NObeyesdad"]

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [48]:
import xgboost as xgb
from xgboost import XGBClassifier, XGBRegressor
from sklearn.metrics import accuracy_score

In [49]:
model_1 = XGBClassifier()
model_1.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [50]:
# make predictions for test data
y_pred = model_1.predict(X_test)
predictions = [round(value) for value in y_pred]

In [51]:
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 90.17%


In [132]:
params = {
    'booster': 'gbtree',
    'eta': 0.05,
    'gamma': 0.01,
    'max_depth': 10,
    'min_child_weight': 7,
    'n_estimators':200,
    'subsample': 0.5,
    'alpha': 0.04,
    'objective': 'multi:softmax',
}

model_2 = XGBClassifier(**params)
model_2.fit(X_train, y_train)

y_pred_2 = model_2.predict(X_test)
predictions_2 = [round(value) for value in y_pred_2]

accuracy_2 = accuracy_score(y_test, predictions_2)
print("Accuracy: %.2f%%" % (accuracy_2 * 100.0))

Accuracy: 90.90%


In [106]:
sample_submission =pd.read_csv("sample_submission.csv")

In [107]:
final_predictions = model_2.predict(df_test_up)
final_prediction_values = [round(value) for value in final_predictions]

In [108]:
len(final_predictions)

13840

In [109]:
import numpy as np

def change_back_to_categories(final_values):
    category_mapping = {
        0: 'Insufficient_Weight',
        1: 'Normal_Weight',
        2: 'Overweight_Level_I',
        3: 'Overweight_Level_II',
        4: 'Obesity_Type_I',
        5: 'Obesity_Type_II',
        6: 'Obesity_Type_III'
    }

    return np.vectorize(category_mapping.get)(final_values)

In [110]:
final_values = final_prediction_values.copy()
final_preds = change_back_to_categories(final_values)
final_preds

array(['Obesity_Type_II', 'Overweight_Level_I', 'Obesity_Type_III', ...,
       'Insufficient_Weight', 'Normal_Weight', 'Obesity_Type_II'],
      dtype='<U19')

In [111]:
sample_submission["NObeyesdad"].value_counts()

NObeyesdad
Normal_Weight    13840
Name: count, dtype: int64

In [112]:
sample_submission["NObeyesdad"] = final_preds
sample_submission

,id,NObeyesdad
0,20758,Obesity_Type_II
1,20759,Overweight_Level_I
2,20760,Obesity_Type_III
3,20761,Obesity_Type_I
4,20762,Obesity_Type_III
...,...,...
13835,34593,Overweight_Level_II
13836,34594,Normal_Weight
13837,34595,Insufficient_Weight
13838,34596,Normal_Weight


In [113]:
sample_submission.to_csv("submission.csv", index=False)
sample_submission.head()

,id,NObeyesdad
0,20758,Obesity_Type_II
1,20759,Overweight_Level_I
2,20760,Obesity_Type_III
3,20761,Obesity_Type_I
4,20762,Obesity_Type_III
